# Generating Multiple Sequence Alignement given PDB structure

Given a PDB structure file we use ProDy to search all available Pfam alignments for the best matching Multiple Sequence Alignment (MSA)

In [12]:
# Initial Import

# System packages
import os.path, sys
from pathlib import Path


# Scientific Computing packages
import numpy as np
np.random.seed(1) # set random seed for reproducibility
import pandas as pd
from scipy import linalg
from scipy.sparse import csr_matrix
from sklearn.preprocessing import OneHotEncoder
from scipy.spatial import distance_matrix


# Biopython packages
import Bio.PDB, warnings
pdb_list = Bio.PDB.PDBList()
pdb_parser = Bio.PDB.PDBParser()
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)
from Bio import SeqIO, pairwise2

# Parallelization and computing diagnosti packages
from joblib import Parallel, delayed
import timeit

# Plotting packages
import matplotlib.pyplot as plt

# # --- Import our Code ---# #
# Direct Coupling Analysis code
from direct_info import direct_info

# Import data processing and general DCA_ER tools
from data_processing import data_processing, pdb2msa
import tools
# # -----------------------# #

# Import ProDy code.
from prody import *

# Define Computational Resources
n_cpus = 10

## Define Data directories
* data_path: Where is the MSA and pdb structure data
* out_dir: Where do you want the processed MSA data to go
* pdb_dir: Where do you want the PDB-MSA reference and structure data to go

In [13]:
data_path = Path('/data/cresswellclayec/DCA_ER/Pfam-A.full')
data_path = Path('/data/cresswellclayec/Pfam-A.full')

# Define data directories
DCA_ER_dir = '/data/cresswellclayec/ExpectationReflection_DCA' # Set DCA_ER directory

# We make the directory and sub-directory to store data.
out_dir = "%s/protein_data" % DCA_ER_dir
processed_data_dir = "%s/data_processing_output" % out_dir
pdb_dir = '%s/pdb_data/' % out_dir

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    os.makedirs(processed_data_dir)
    os.makedirs(pdb_dir)

pfam_dir = "/fdb/fastadb/pfam"

# Path to gzipped pdb file
pdb_path = "/pdb/pdb/zd/pdb1zdr.ent.gz"


## Un-zip and load PDB file to get PDB structure (1zdr)

In [14]:
import gzip, shutil
def gunzip(file_path, output_path):
    print('Unzipping %s to %s' % (file_path, output_path))
    with gzip.open(file_path,"rb") as f_in, open(output_path,"wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

        
        
unzipped_pdb_filename = os.path.basename(pdb_path).replace(".gz", "")
pdb_out_path = "%s%s" % (pdb_dir, unzipped_pdb_filename)
gunzip(pdb_path, pdb_out_path)

Unzipping /pdb/pdb/zd/pdb1zdr.ent.gz to /data/cresswellclayec/ExpectationReflection_DCA/protein_data/pdb_data/pdb1zdr.ent


## Generate PDB2MSA Data
* For each available polypeptide sequence in the PDB structure us ProDy to BLAST for corresponding sequences in the Pfam domain

In [16]:
pdb2msa_results = pdb2msa(pdb_out_path, pdb_dir, create_new=False)
print('PDB2MSA Results:\n', pdb2msa_results)

if len(pdb2msa_results) > 1:
    fasta_file = pdb2msa_results[0]
    prody_df = pdb2msa_results[1]
else:
    prody_df = pdb2msa_results[0]


print('\n\n\nPDB DF with associated Protein Families\n', prody_df.loc[:,  [column for column in prody_df.columns if column not in ['locations', 'PDB Sequence']]].head())

PDB2MSA Results:
 [   Unnamed: 0 PDB ID Chain  Polypeptide Index     Pfam   accession   class  \
0           0   1zdr     B                  0  PF00186  PF00186.22  Domain   
1           1   1zdr     A                  0  PF00186  PF00186.22  Domain   

       id    type                                       PDB Sequence  ali_end  \
0  DHFR_1  Pfam-A  MISHIVAMDENRVIGKDNRLPWHLPADLAYFKRVTMGHAIVMGRKT...      160   
1  DHFR_1  Pfam-A  MISHIVAMDENRVIGKDNRLPWHLPADLAYFKRVTMGHAIVMGRKT...      160   

   ali_start  bitscore  end   cond_evalue    ind_evalue    evidence  hmm_end  \
0          1    215.11  160  2.300000e-68  4.500000e-64  hmmer v3.0      160   
1          1    215.08  160  2.300000e-68  4.600000e-64  hmmer v3.0      160   

   hmm_start  start  
0          1      1  
1          1      1  ]



PDB DF with associated Protein Families
    Unnamed: 0 PDB ID Chain  Polypeptide Index     Pfam   accession   class  \
0           0   1zdr     B                  0  PF00186  PF00186.22  Doma

## Load and Pre-Process Pfam-MSA
* With the PDB2MSA data we can now loop through all the different PDB-MSA pairings for all available sequences
* Because the PDB2MSA data is ordered by PDB-polypeptide sequence to MSA alignment score the we 

In [17]:
print("\n\n\nLooping through Prody Search DataFrame:\n", prody_df.head())
rows_to_drop = []
for ir, pdb2msa_row in enumerate(prody_df.iterrows()):
    print('\n\nGetting msa with following pdb2msa entry:\n', pdb2msa_row)
    #try:
    dp_result =  data_processing(data_path, prody_df.iloc[pdb2msa_row[0]], gap_seqs=0.2, gap_cols=0.2, prob_low=0.004,
                               conserved_cols=0.8, printing=True, out_dir=processed_data_dir, pdb_dir=pdb_dir, letter_format=False,
                               remove_cols=True, create_new=True, n_cpu=min(2, n_cpus))
    if dp_result is not None:
        [s0, removed_cols, s_index, tpdb, pdb_s_index] = dp_result
        break
    else:
        rows_to_drop.append(ir)
        continue



pdb_id = pdb2msa_row[1]['PDB ID']
pfam_id = pdb2msa_row[1]['Pfam']
# update Prody search DF (use same filename as pdb2msa() in data_processing
prody_df = prody_df.drop(rows_to_drop)
print("\n\n\nSaving updated Prody Search DataFrame:\n", prody_df.head())
prody_df.to_csv('%s/%s_pdb_df.csv' % (pdb_dir, pdb_id))

if dp_result is None:
    print('None of the available prody pdb search found matching alignments... Exiting..')
    sys.exit()
print('Done Preprocessing Data.....')



# number of positions
n_var = s0.shape[1]
n_seq = s0.shape[0]

print("Number of residue positions:",n_var)
print("Number of sequences:",n_seq)




Looping through Prody Search DataFrame:
    Unnamed: 0 PDB ID Chain  Polypeptide Index     Pfam   accession   class  \
0           0   1zdr     B                  0  PF00186  PF00186.22  Domain   
1           1   1zdr     A                  0  PF00186  PF00186.22  Domain   

       id    type                                       PDB Sequence  ali_end  \
0  DHFR_1  Pfam-A  MISHIVAMDENRVIGKDNRLPWHLPADLAYFKRVTMGHAIVMGRKT...      160   
1  DHFR_1  Pfam-A  MISHIVAMDENRVIGKDNRLPWHLPADLAYFKRVTMGHAIVMGRKT...      160   

   ali_start  bitscore  end   cond_evalue    ind_evalue    evidence  hmm_end  \
0          1    215.11  160  2.300000e-68  4.500000e-64  hmmer v3.0      160   
1          1    215.08  160  2.300000e-68  4.600000e-64  hmmer v3.0      160   

   hmm_start  start  
0          1      1  
1          1      1  


Getting msa with following pdb2msa entry:
 (0, Unnamed: 0                                                           0
PDB ID                                            

FileNotFoundError: [Errno 2] No such file or directory: '/data/cresswellclayec/ExpectationReflection_DCA/protein_data/data_processing_output/PF00186_1zdr_preproc_msa.npy'